## Agents

In [ ]:
import pandas as pd

def generate_banking_data():
    # Sample account-level data
    account_data = {
        'customer_name': ['Alice', 'Bob', 'Charlie', 'David'],
        'customer_id': [1, 2, 3, 4],
        'pr_number': ['PR001', 'PR002', 'PR003', 'PR004'],
        'income': [50000, 60000, 70000, 80000],
        'age': [30, 40, 35, 50]
    }

    # Sample transaction-level data
    transaction_data = {
        'ucic_id': [101, 102, 103, 104, 105, 106],
        'customer_id': [1, 1, 2, 3, 4, 4],
        'category': ['electronics', 'investments', 'investments', 'groceries', 'investments', 'electronics'],
        'transaction_amount': [200, 1500, 2000, 300, 2500, 400],
        'credit_debit': ['debit', 'credit', 'credit', 'debit', 'credit', 'debit']
    }

    # Create DataFrames
    account_df = pd.DataFrame(account_data)
    transaction_df = pd.DataFrame(transaction_data)

    return account_df, transaction_df

# Example usage
account_df, transaction_df = generate_banking_data()
print("Account Level Info:")
print(account_df)
print("\nTransaction Level Info:")
print(transaction_df)


from crewai import Agent

# Initialize CrewAI agent
data_analyzer = Agent(
    role='Banking Data Analyst',
    goal='Analyze and compare investment data with peers',
    verbose=True,
    memory=True,
    backstory='Expert in banking data analysis, focusing on customer transactions and peer comparisons.',
    tools=[],  # Define any tools if necessary
    allow_delegation=False
)

def analyze_investments(account_df, transaction_df):
    # Filter for investment transactions
    investment_transactions = transaction_df[transaction_df['category'] == 'investments']

    # Calculate total investments per customer
    investment_summary = investment_transactions.groupby('customer_id').sum(numeric_only=True)

    # Merge with account info for peer comparison
    investment_comparison = investment_summary.merge(account_df, on='customer_id', how='left')

    return investment_comparison

# Example usage of analysis
investment_comparison = analyze_investments(account_df, transaction_df)
print("\nInvestment Comparison with Peers:")
print(investment_comparison)

Account Level Info:
  customer_name  customer_id pr_number  income  age
0         Alice            1     PR001   50000   30
1           Bob            2     PR002   60000   40
2       Charlie            3     PR003   70000   35
3         David            4     PR004   80000   50

Transaction Level Info:
   ucic_id  customer_id     category  transaction_amount credit_debit
0      101            1  electronics                 200        debit
1      102            1  investments                1500       credit
2      103            2  investments                2000       credit
3      104            3    groceries                 300        debit
4      105            4  investments                2500       credit
5      106            4  electronics                 400        debit

Investment Comparison with Peers:
   customer_id  ucic_id  transaction_amount customer_name pr_number  income  \
0            1      102                1500         Alice     PR001   50000   
1           

In [ ]:
from crewai_tools import CustomInvestmentAnalysisTool

# Initialize the tool for analyzing investment patterns
investment_tool = CustomInvestmentAnalysisTool(
    description='Tool for analyzing customer investments and comparing with market trends',
    data_source='Customer Transaction Data'  # Customize this as per your data source or requirement
)

# Pass the tool to the agent
data_analyzer = Agent(
    role='Banking Data Analyst',
    goal='Analyze and compare investment data with peers',
    verbose=True,
    memory=True,
    backstory='Expert in banking data analysis, focusing on customer transactions and peer comparisons.',
    tools=[investment_tool],  # Add the custom tool here
    allow_delegation=False
)


ModuleNotFoundError: No module named 'crewai_tools'

## Crew

In [ ]:
from crewai import Crew, Process
from crewai import Agent  # Assuming Agent needs to be imported here

# Define the banking data analyst agent
data_analyzer = Agent(
    role='Banking Data Analyst',
    goal='Analyze and compare investment data with peers',
    verbose=True,
    memory=True,
    backstory='Expert in banking data analysis, focusing on customer transactions and peer comparisons.',
    tools=[],  # Define any tools if necessary
    allow_delegation=False
)

# Define the task to analyze investments
def analyze_investments_task(account_df, transaction_df):
    def task(inputs):
        return analyze_investments(account_df, transaction_df)
    return task

# Create the crew with the banking data analyst agent
crew = Crew(
    agents=[data_analyzer],
    tasks=[analyze_investments_task],  # Updated task to analyze investments
    process=Process.sequential,  # Optional: Sequential task execution is default
    memory=True,
    cache=True,
    max_rpm=100,
    share_crew=True
)

# Generate sample banking data
account_df, transaction_df = generate_banking_data()

# Start the task execution process
result = crew.kickoff(inputs={'account_df': account_df, 'transaction_df': transaction_df})
print(result)


AttributeError: 'function' object has no attribute 'get'

##